# Gather Hourly Traffic Data <br>
*Author(s): Anton Olav Glad & Filip Moen Nilsen* <br>
 <br>
This Jupyter Notebook shoes step by step how to extract **hourly** data from traffic points throughout the Norwegian Road Network. <br>
The API and documentation is available at: *https://www.vegvesen.no/trafikkdata/api* <br>
The first query aims to extract relevant traffic point IDs, which is later on used as a input variable in the second queries. <br>
<br>
**NOTE:** There is only possible to get 100 observations from each volume-query. Therefore Query #2 is evenly split into two queries. <br>

Import necessary packages:

In [1]:
import os
import json
import requests
import time
import random
from tqdm import tqdm  
import pandas as pd
from datetime import datetime
# Display all columns in the df
pd.set_option('display.max_columns', None)

Set paths etc.:

In [2]:
# define the base path where the file should be saved
base_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData'
# define the file name
file_name = "all_IDs.json"
# combine the base path and file name to create a valid file path
file_path = os.path.join(base_path, file_name)

### First query (get traffic point ID's):

In [3]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

# define the GraphQL query for fetching ID's of registration points (only Europe roads)
query = """
{
  trafficRegistrationPoints(searchQuery: {roadCategoryIds: [E ,R, F], registrationFrequency: CONTINUOUS}) {
    id
  }
}
"""

# make a POST request to the API
response = requests.post(endpoint, json={"query": query})

# check the status code of the response
if response.status_code == 200:
    # if the request was successful, get the data from the response
    data = response.json()

    # open the file for writing
    with open(file_path, "w") as file:
        # write the data to the file in JSON format
        json.dump(data, file)

Store query number 1 and convert ID's to list for inserting in query number 2:

In [4]:
# define the file path of the JSON file
file_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/all_IDs.json'

# open the file for reading
with open(file_path, "r") as file:
    # load the data from the file into a Python object
    data = json.load(file)

# create an empty list for the registration point ID's
registration_point_ids = []

# iterate over the data returned by the GraphQL query
for item in data["data"]["trafficRegistrationPoints"]:
    # get the ID of the current registration point
    id = item["id"]

    # add the ID to the list of registration point ID's
    registration_point_ids.append(id)

### Second query (get traffic volume for every hour in a one-week period): <br>
*NB! This query must be run **twice** because it only possible to get 100 hours per query.* <br>
*The dataframes must therefore be merged together later on.* <br>
**WARNING:** Takes a *looong* time. About 2 hours per query. Better run this before going to bed. 

In [5]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

data_1 = []
with tqdm(total=len(registration_point_ids)) as pbar:
  for registration_point_id in registration_point_ids:
    time.sleep(random.uniform(1, 3))

    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-03-26T00:00:00+02:00"
        to: "2018-03-29T11:00:00+02:00"  
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_1 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_1["registration_point_id"] = registration_point_id

      data_1.append(volume_data_1)

      pbar.update(1)

100%|██████████| 3126/3126 [1:49:31<00:00,  2.10s/it]


*Second run:*

In [6]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

data_2 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-03-29T11:00:00+02:00"
        to: "2018-04-01T00:00:00+02:00"
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_2 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_2["registration_point_id"] = registration_point_id

      data_2.append(volume_data_2)

      pbar.update(1)

100%|█████████▉| 3125/3126 [1:49:27<00:02,  2.10s/it]  


Store data collected from query number 2 (the two queries above):

In [7]:
# define the path and filename for the JSON file
filename_1 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1_Easter.json'
# write the data to the JSON file
with open(filename_1, "w") as f:
    json.dump(data_1, f, indent=2) # usually indent = 4, but ok 

# define the path and filename for the JSON file
filename_2 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2_Easter.json'
# write the data to the JSON file
with open(filename_2, "w") as f:
    json.dump(data_2, f, indent=2) 

Open data collected from query number 2:

In [8]:
with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1_Easter.json', 'r') as file:
    json_data_1 = json.load(file)

with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2_Easter.json', 'r') as file:
    json_data_2 = json.load(file)

### def flatten_json <br>
*Takes the data gathered from query 2 (stored as a json-file), flattening the data and converts it to a pandas dataframe.*

In [40]:
def flatten_json(json_data):
    flat_list = []
    for item in json_data:
        edges = item.get("data", {}).get("trafficData", {}).get("volume", {}).get("byHour", {}).get("edges", [])
        traffic_registration_point_id = item.get("registration_point_id")

        data_count = 0
        temp_list = []
        for edge in edges:
            node = edge.get("node", {})
            total = node.get("total", {})

            if total is None:
                continue

            volume_numbers = total.get("volumeNumbers") or {}
            coverage = total.get("coverage") or {}

            if volume_numbers.get("volume") is None:
                continue

            from_datetime = datetime.fromisoformat(node.get("from").replace("+02:00", ""))
            to_datetime = datetime.fromisoformat(node.get("to").replace("+02:00", ""))

            flat_dict = {
                "traffic_point_id": traffic_registration_point_id,
                "year": from_datetime.year,
                "month": from_datetime.month,
                "day": from_datetime.day,
                "from_hour": from_datetime.strftime("%H:%M"),
                "to_hour": to_datetime.strftime("%H:%M"),
                "volume": volume_numbers.get("volume"),
                "coverage_percentage": coverage.get("percentage")
            }
            data_count += 1
            temp_list.append(flat_dict)

        if data_count >= 61:    # 83 er riktig tall egentlig, men denne kan senkes for å se hvor mange traffikpunkter som mangler verdi minst én time på dagen. Dette traffikpunktet vil da IKKE bli inkludert. 
            flat_list.extend(temp_list)

    return flat_list


**Read warning in function above.**

Run function, check output:

In [41]:
flat_data_1 = flatten_json(json_data_1)
flat_data_2 = flatten_json(json_data_2)
# Convert flattened data to DataFrame
volume_df_1 = pd.DataFrame(flat_data_1)
volume_df_2 = pd.DataFrame(flat_data_2)
display(volume_df_1)#[:50]
display(volume_df_2)#[:50]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,3,26,00:00,01:00,11,100.0
1,65823V1668921,2018,3,26,01:00,02:00,4,100.0
2,65823V1668921,2018,3,26,02:00,03:00,3,100.0
3,65823V1668921,2018,3,26,03:00,04:00,5,100.0
4,65823V1668921,2018,3,26,04:00,05:00,8,100.0
...,...,...,...,...,...,...,...,...
153830,10795V320297,2018,3,29,06:00,07:00,51,100.0
153831,10795V320297,2018,3,29,07:00,08:00,91,100.0
153832,10795V320297,2018,3,29,08:00,09:00,91,100.0
153833,10795V320297,2018,3,29,09:00,10:00,139,100.0


,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,3,29,11:00,12:00,48,100.0
1,65823V1668921,2018,3,29,12:00,13:00,50,100.0
2,65823V1668921,2018,3,29,13:00,14:00,62,100.0
3,65823V1668921,2018,3,29,14:00,15:00,82,100.0
4,65823V1668921,2018,3,29,15:00,16:00,72,100.0
...,...,...,...,...,...,...,...,...
110832,10795V320297,2018,3,31,19:00,20:00,353,100.0
110833,10795V320297,2018,3,31,20:00,21:00,271,100.0
110834,10795V320297,2018,3,31,21:00,22:00,231,100.0
110835,10795V320297,2018,3,31,22:00,23:00,204,100.0


### Merge the two dataframes:

In [54]:
# Concatenate the two dataframes
merged_df = pd.concat([volume_df_1, volume_df_2], ignore_index=True)

# Group by traffic_point_id and filter based on the number of data points (168 hours)
completeVolume_df = merged_df.groupby('traffic_point_id').filter(lambda x: len(x) >= 144)

# Reset the index of the filtered dataframe
completeVolume_df.reset_index(drop=True, inplace=True)
completeVolume_df#[:60]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,3,26,00:00,01:00,11,100.0
1,65823V1668921,2018,3,26,01:00,02:00,4,100.0
2,65823V1668921,2018,3,26,02:00,03:00,3,100.0
3,65823V1668921,2018,3,26,03:00,04:00,5,100.0
4,65823V1668921,2018,3,26,04:00,05:00,8,100.0
...,...,...,...,...,...,...,...,...
256603,10795V320297,2018,3,31,19:00,20:00,353,100.0
256604,10795V320297,2018,3,31,20:00,21:00,271,100.0
256605,10795V320297,2018,3,31,21:00,22:00,231,100.0
256606,10795V320297,2018,3,31,22:00,23:00,204,100.0


Check the merged dataframe - looks good:

Save dataframe as CSV-file in directory:

In [55]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_Easter.csv"

# Convert the dataframe to a CSV file and save it to the specified path
completeVolume_df.to_csv(output_file_path, index=False)


In [56]:
print(completeVolume_df['traffic_point_id'].nunique() / len(registration_point_ids))

0.5700575815738963


In [15]:
#completeVolume_df = pd.read_csv('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_July.csv')
#completeVolume_df

### Pivot the dataframe <br> 
*This is to display hourly volume on the y-axis (not sure if necessary)*

In [57]:
# Create a new column 'timestamp' combining year, month, day, and from_hour
completeVolume_df['timestamp'] = completeVolume_df['year'].astype(str) + '-' + completeVolume_df['month'].astype(str).str.zfill(2) + '-' + completeVolume_df['day'].astype(str).str.zfill(2) + ' ' + completeVolume_df['from_hour']

# Pivot the DataFrame using 'traffic_point_id' as index and 'timestamp' as columns
pivoted_df = completeVolume_df.pivot_table(index='traffic_point_id', columns='timestamp', values='volume')

# Reset index and fill missing values with 0 (if any)
pivoted_df.reset_index(inplace=True)
pivoted_df.fillna(0, inplace=True)

# Display pivoted dataframe
pivoted_df

timestamp,traffic_point_id,2018-03-26 00:00,2018-03-26 01:00,2018-03-26 02:00,2018-03-26 03:00,2018-03-26 04:00,2018-03-26 05:00,2018-03-26 06:00,2018-03-26 07:00,2018-03-26 08:00,2018-03-26 09:00,2018-03-26 10:00,2018-03-26 11:00,2018-03-26 12:00,2018-03-26 13:00,2018-03-26 14:00,2018-03-26 15:00,2018-03-26 16:00,2018-03-26 17:00,2018-03-26 18:00,2018-03-26 19:00,2018-03-26 20:00,2018-03-26 21:00,2018-03-26 22:00,2018-03-26 23:00,2018-03-27 00:00,2018-03-27 01:00,2018-03-27 02:00,2018-03-27 03:00,2018-03-27 04:00,2018-03-27 05:00,2018-03-27 06:00,2018-03-27 07:00,2018-03-27 08:00,2018-03-27 09:00,2018-03-27 10:00,2018-03-27 11:00,2018-03-27 12:00,2018-03-27 13:00,2018-03-27 14:00,2018-03-27 15:00,2018-03-27 16:00,2018-03-27 17:00,2018-03-27 18:00,2018-03-27 19:00,2018-03-27 20:00,2018-03-27 21:00,2018-03-27 22:00,2018-03-27 23:00,2018-03-28 00:00,2018-03-28 01:00,2018-03-28 02:00,2018-03-28 03:00,2018-03-28 04:00,2018-03-28 05:00,2018-03-28 06:00,2018-03-28 07:00,2018-03-28 08:00,2018-03-28 09:00,2018-03-28 10:00,2018-03-28 11:00,2018-03-28 12:00,2018-03-28 13:00,2018-03-28 14:00,2018-03-28 15:00,2018-03-28 16:00,2018-03-28 17:00,2018-03-28 18:00,2018-03-28 19:00,2018-03-28 20:00,2018-03-28 21:00,2018-03-28 22:00,2018-03-28 23:00,2018-03-29 00:00,2018-03-29 01:00,2018-03-29 02:00,2018-03-29 03:00,2018-03-29 04:00,2018-03-29 05:00,2018-03-29 06:00,2018-03-29 07:00,2018-03-29 08:00,2018-03-29 09:00,2018-03-29 10:00,2018-03-29 11:00,2018-03-29 12:00,2018-03-29 13:00,2018-03-29 14:00,2018-03-29 15:00,2018-03-29 16:00,2018-03-29 17:00,2018-03-29 18:00,2018-03-29 19:00,2018-03-29 20:00,2018-03-29 21:00,2018-03-29 22:00,2018-03-29 23:00,2018-03-30 00:00,2018-03-30 01:00,2018-03-30 02:00,2018-03-30 03:00,2018-03-30 04:00,2018-03-30 05:00,2018-03-30 06:00,2018-03-30 07:00,2018-03-30 08:00,2018-03-30 09:00,2018-03-30 10:00,2018-03-30 11:00,2018-03-30 12:00,2018-03-30 13:00,2018-03-30 14:00,2018-03-30 15:00,2018-03-30 16:00,2018-03-30 17:00,2018-03-30 18:00,2018-03-30 19:00,2018-03-30 20:00,2018-03-30 21:00,2018-03-30 22:00,2018-03-30 23:00,2018-03-31 00:00,2018-03-31 01:00,2018-03-31 02:00,2018-03-31 03:00,2018-03-31 04:00,2018-03-31 05:00,2018-03-31 06:00,2018-03-31 07:00,2018-03-31 08:00,2018-03-31 09:00,2018-03-31 10:00,2018-03-31 11:00,2018-03-31 12:00,2018-03-31 13:00,2018-03-31 14:00,2018-03-31 15:00,2018-03-31 16:00,2018-03-31 17:00,2018-03-31 18:00,2018-03-31 19:00,2018-03-31 20:00,2018-03-31 21:00,2018-03-31 22:00,2018-03-31 23:00
0,00000V1702725,10,2,2,5,7,19,100,294,306,400,752,1041,1174,1216,1313,1455,1481,1199,959,838,633,303,46,14,10,1,1,5,4,18,99,294,263,415,795,1035,1169,1256,1342,1450,1401,1231,1024,832,643,320,52,12,10,2,2,2,3,21,80,262,228,407,873,1312,1585,1611,1568,1440,1305,1006,826,629,497,282,65,13,10,6,0,3,0,2,6,10,13,27,45,45,99,145,147,144,142,109,65,56,50,34,26,16,5,0,2,3,3,0,2,5,12,37,38,68,110,104,127,137,99,87,81,70,85,102,66,54,31,8,1,3,3,4,14,33,62,339,869,1226,1422,1348,1239,1127,737,151,88,41,45,33,18,10
1,00000V1702751,172,74,55,50,118,387,1196,1979,1951,1936,2563,2995,3342,3396,3542,4191,3583,2824,2241,1855,1441,970,502,304,190,115,84,65,117,363,1213,1859,1745,1904,2455,3084,3209,3340,3629,4033,3538,2900,2289,1899,1461,987,552,317,261,108,91,60,126,358,1011,1541,1374,1708,2565,3271,3760,3744,3672,3414,3122,2567,1988,1594,1253,850,569,311,262,103,77,50,54,137,285,279,332,492,849,1074,1186,1306,1445,1405,1328,1286,914,715,493,472,314,256,181,110,72,61,53,95,199,220,192,445,781,1061,1213,1268,1461,1472,1291,1153,920,755,699,590,351,276,256,100,72,45,56,130,230,265,387,983,1969,2703,3037,2857,2684,2404,2022,1092,863,779,595,407,387,320
2,00000V1993681,66,22,14,32,92,215,761,1131,950,644,556,575,601,644,702,939,898,606,462,400,331,276,170,122,55,33,18,29,84,175,748,1043,798,507,495,493,545,649,666,905,742,606,443,363,346,250,174,116,89,31,23,32,60,186,546,757,534,441,530,644,685,689,534,655,486,480,403,298,245,163,174,116,74,37,24,45,49,77,104,172,179,229,205,282,326,422,366,379,341,326,284,236,182,159,1

Store new pivoted dataframe:

In [58]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_Easter_pivoted.csv"

# Convert the dataframe to a CSV file and save it to the specified path
pivoted_df.to_csv(output_file_path, index=False)